In [1]:
#Bibliotheken:
from bs4 import BeautifulSoup
from lxml import etree as ET
import pandas as pd
import unicodedata
import polars as pl

In [2]:
#Read df from file:
dump =  pl.read_parquet('data/online_diss_03-2024.parquet') 
df = dump.to_pandas()


In [3]:


#df =  pd.read_hdf('data/online-diss_03-2024.h5', 'df') 
find_none = df.loc[df['diss'] == "not found"]

#df =  pl.read_parquet('data/online_diss_03-2024.parquet') 
#find_none = df.filter(pl.col('diss') == 'not found')
no_thesis = (len(find_none))
print("Anzahl der Datensätze, die keinen Hochschulschriftenvermerk enthalten: ", no_thesis)

Anzahl der Datensätze, die keinen Hochschulschriftenvermerk enthalten:  576


In [4]:
print(len(df))

325358


In [5]:
#Drop all records that don't have a dissertation statement:
df['diss'] = df['diss'].replace(["not found"], [None])
df = df.dropna(subset=['diss'])
df

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008
190,1000000389,"Klöfkorn, Robert",Numerics for evolution equations : a general i...,eng,"Freiburg (Breisgau), Univ., Diss., 2009","Freiburg (Breisgau), Univ., Diss., 2009",N/A,N/A,N/A,510,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
191,1000006395,"Speckmann, Bodo",Regulation der hepatischen Biosynthese von Sel...,ger,"Düsseldorf, Univ., Diss., 2009","Düsseldorf, Univ., Diss., 2009",570,570,DE-101,573.3845554,540,"Düsseldorf, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
192,100000645X,"Sauheitl, Leopold",Direct amino acid uptake by plants related to ...,eng,"Bayreuth, Univ., Diss., 2009","Bayreuth, Univ., Diss., 2009",N/A,N/A,N/A,570,N/A,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
193,1000006948,"Sanchez, Martin",Charakterisierung des Replikationsenzyms ORF90...,ger,"Bayreuth, Univ., Diss., 2010","Bayreuth, Univ., Diss., 2010",570,570,DE-101,572.86453629321,N/A,"Bayreuth, Univ., Diss., 2010",N/A,100204s2009 gw |||||om||| 00||||ger
194,1000007138,"Reimann, Bodo",Numerische Untersuchung der Stoßrohrströmung ...,ger,"Duisburg, Essen, Univ., Diss., 2009","Duisburg, Essen, Univ., Diss., 2009",620,620,DE-101,629.4152,620,"Duisburg, Essen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325353,999997297,"Moschkau, Peter",Defektabhängige Transporteigenschaften von Pr...,ger,"Göttingen, Univ., Diss., 2009","Göttingen, Univ., Diss., 2009",N/A,N/A,N/A,530,N/A,"Göttingen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
325354,999997505,"Scheler, Esther",Tailoring fluorene-based oligomers for micron ...,eng,"Bayreuth, Univ., Diss., 2009","Bayreuth, Univ., Diss., 2009",540,540,DE-101,547.70455,VN 5927,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
325355,99999767X,"Körzdörfer, Thomas",Self-interaction and charge transfer in organi...,eng,"Bayreuth, Univ., Diss., 2009","Bayreuth, Univ., Diss., 2009",530,530,DE-101,537.6223,UM 1200,"Bayreuth, Univ., Diss., 2009",Enth. 4 Sonderabdr. aus verschiedenen Zeitschr.,100204s2009 gw |||||om||| 00||||eng
325356,999999389,"Xystrakis, Fotios",The drought tolerance limit of European beec...,eng,"Freiburg (Breisgau), Univ., Diss., 2009","Freiburg (Breisgau), Univ., Diss., 2009",N/A,N/A,N/A,630,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",Dateien in unterschiedlichen Formaten,100204s2009 gw |||||om||| 00||||eng


In [6]:
df.filter(pl.col('Place') == 'N/A')

TypeError: Index(...) must be called with a collection of some kind, <Expr ['[(col("Place")) == (String(N/A…'] at 0x1ECB2EBE210> was passed

In [ ]:
#Now clean columns "Place" and "Year":

df['Place'] = df['Place'].str.replace(',.*', '', regex=True) #For records with disseratation statement in "Place" keep string until first comma 
df['Place'] = df['Place'].str.replace("Zugl.: ", "")
df['Year'] = df['Year'].str.extract('(\d+)')
df


In [7]:
#df.to_hdf('data/online-diss-cleaned_03-2024.h5', key='df')

In [8]:
cleaned_dump = pl.from_pandas(df)

In [ ]:
cleaned_dump.write_parquet("data/online_diss_cleaned_03-2024.parquet")